In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
import gc

In [2]:
dtypes = {
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'hour'          : 'uint8',
        'is_attributed' : 'uint8', 
        'ip_day_hour_count': 'uint32', 
        'ip_os_day_hour_count': 'uint32', 
        'ip_app_day_hour_count': 'uint32', 
        'ip_app_os_day_hour_count': 'uint32', 
        'app_day_hour_count': 'uint32', 
        'ip_device_os_count': 'uint32', 
        'ip_app_device_os_count': 'uint32', 
        'ip_device_os_mean': 'float16',
        'ip_app_device_os_mean': 'float16',
        'ip_app_device_mean': 'float16',
        'app_device_os_mean': 'float16',
        'ip_device_os_time2nextclick': 'int32',
        'ip_app_device_os_time2nextclick': 'int32',
        'ip_app_device_time2nextclick': 'int32',
        'ip_device_os_time2previousclick': 'int32',
        'ip_app_device_os_time2previousclick': 'int32',
        'ip_app_device_time2previousclick': 'int32',
        'ip_device_os_countfromfuture': 'uint32', 
        'ip_app_device_os_countfromfuture': 'uint32', 
        'ip_app_device_countfromfuture': 'uint32', 
        'ip_device_os_countfrompast': 'uint32', 
        'ip_app_device_os_countfrompast': 'uint32', 
        'ip_app_device_countfrompast': 'uint32', 
        'ip_device_os_lasttimediff': 'int32',
        'ip_app_device_os_lasttimediff': 'int32',
        'ip_app_device_lasttimediff': 'int32',
        'ip_device_os_firsttimediff': 'int32',
        'ip_app_device_os_firsttimediff': 'int32',
        'ip_app_device_firsttimediff': 'int32',
        'matrixFact_user_iposdeviceapp_item_app': 'float16',
        'matrixFact_user_ip_item_appdeviceos': 'float16',
        'matrixFact_user_ipchannel_item_appdeviceos': 'float16',
        'ip_device_os_regression': 'float16',
        'ip_app_device_os_regression': 'float16',
        'ip_app_device_regression': 'float16',
        'ip_app_device_os_channel_regression': 'float16', 
        'attributed_timediffmax':'int32',
        'attributed_timediffmin':'int32',
        'attributed_timediff':'float16',
        'matrixFact_user_ipappdeviceos_item_channel': 'float16'
    
        } 

# Load Data

In [ ]:
df_all = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/equalhour/ALL_features_supplementV3_feature42.csv',dtype=dtypes)
target = 'is_attributed'
extra = ['ip_app_device_os_channel_regression']
feature_cols = list(set(df_all.columns) - set([target]) -set(extra))


# Train XGBoost

In [ ]:
params = {'learning_rate': 0.03, 
         'max_depth':7, 
         'nthread':90,
         'seed':1, 
         'silent':False, 
         'n_estimators':1200, 
         'scale_pos_weight':99
        }

In [ ]:
def train_xgboost(x_train, x_val, feature_cols, category_index, params, best_round = None, target='is_attributed'):
    param = params.copy()    
    print('Start training')
    model = xgb.XGBClassifier(**param)
    model.fit(x_train[feature_cols].values, x_train[target].values,
          eval_set=[(x_train[feature_cols].values[-1000000:], x_train[target].values[-1000000:])], 
          eval_metric ='auc',
          verbose=True)
    return model

# Train and Save Mode

In [ ]:
save_model_name = '/home/kai/talkingdata/data/all_suppelement_xgbtree_1200_depth7_scale99_lr0.03.pickle.dat'
model = train_xgboost(df_all, None, feature_cols,None,params,None,target)
pickle.dump(model, open(save_model_name, "wb"))

In [ ]:
model_load = pickle.load(open(save_model_name, "rb"))
c = model_load.predict_proba(valset[feature_cols].values, ntree_limit=89)[:,1]